# Práctica 1: KNN y selección de atributos
Aprendizaje Automático II, 2025-2026

**Integrantes:** Nombre Apellido1 Apellido2 (reemplazar por los nombres del grupo)

Este notebook contiene la resolución de los ejercicios de la Práctica 1.

## 1. Implementación y uso de KNN
En esta sección se implementa y utiliza el algoritmo KNN sobre el dataset de cáncer de mama.

In [ ]:
# a) Descarga de datos
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
X, y = data.data, data.target

Shape X: (569, 30), Shape y: (569,)


In [4]:
# b) Preprocesado: separación, partición y normalización
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Separación ya realizada arriba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(f'Train shape: {X_train.shape}, Test shape: {X_test.shape}')

Train shape: (398, 30), Test shape: (171, 30)


### c) Pregunta teórica: orden de normalización y completado de datos ausentes
**Respuesta:** Si hay datos ausentes, primero se deben completar (imputar) y luego normalizar. Si se normaliza antes de completar, los valores imputados pueden distorsionar la escala. Si se normaliza antes de dividir en train/test, se produce filtrado de información (data leakage).

In [3]:
# d) Importar e instanciar el clasificador KNN personalizado
from KNNClassifier import KNNClassifier
import numpy as np

def euclidean_distance(x1, x2):
    return np.linalg.norm(x1 - x2)

knn = KNNClassifier(n_neighbors=5, distance_func=euclidean_distance)
knn.fit(X_train, y_train)
y_pred_custom = knn.predict(X_test)
print('Predicciones (primeros 10):', y_pred_custom[:10])

TypeError: KNNClassifier.__init__() got an unexpected keyword argument 'n_neighbors'

In [ ]:
# e) Evaluación del modelo KNN personalizado
from sklearn.metrics import accuracy_score
acc_custom = accuracy_score(y_test, y_pred_custom)
print(f'Accuracy KNN personalizado: {acc_custom:.4f}')

### f) Pregunta teórica: desbalanceo de clases
**Respuesta:** Si hay desbalanceo de clases, KNN puede estar sesgado hacia la clase mayoritaria. Una solución es ponderar el voto de los vecinos por la inversa de la distancia o usar técnicas de balanceo como sobremuestreo o submuestreo.

### g) Pregunta teórica: coste en memoria de KNN
**Respuesta:** KNN almacena todo el conjunto de entrenamiento, por lo que el coste en memoria es O(n·d), siendo n el número de muestras y d el número de atributos. Se puede reducir usando técnicas de reducción de datos como prototipos o clustering.

In [ ]:
# h) Comparación con KNeighborsClassifier de sklearn
from sklearn.neighbors import KNeighborsClassifier
knn_sk = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
knn_sk.fit(X_train, y_train)
y_pred_sk = knn_sk.predict(X_test)
acc_sk = accuracy_score(y_test, y_pred_sk)
print(f'Accuracy sklearn KNN: {acc_sk:.4f}')
error_medio = np.mean(y_pred_sk != y_pred_custom)
print(f'Error medio entre predicciones: {error_medio:.4f}')

### i) Pregunta teórica: método alternativo ponderado
**Respuesta:** El método ponderado por la inversa de la distancia puede ser mejor si los vecinos más cercanos son más relevantes, pero es más sensible a outliers. KNN estándar da igual peso a los k vecinos. La elección depende del problema.

## 2. Optimización de KNN
En esta sección se optimiza el número de vecinos y se exploran diferentes métricas de distancia.

In [ ]:
# a) Validación cruzada para encontrar k óptimo
from sklearn.model_selection import cross_val_score
k_range = range(1, 21)
cv_scores = []
for k in k_range:
    knn_cv = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    scores = cross_val_score(knn_cv, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())
k_opt = k_range[np.argmax(cv_scores)]
print(f'k óptimo: {k_opt}')

In [ ]:
# b) Accuracy en test con k óptimo
knn_opt = KNeighborsClassifier(n_neighbors=k_opt, metric='euclidean')
knn_opt.fit(X_train, y_train)
y_pred_opt = knn_opt.predict(X_test)
acc_opt = accuracy_score(y_test, y_pred_opt)
print(f'Accuracy test con k óptimo: {acc_opt:.4f}')

### c) Pregunta teórica: ¿k óptimo en test?
**Respuesta:** El k óptimo encontrado por validación cruzada maximiza la accuracy en training, pero no necesariamente en test, aunque suele ser un buen estimador.

In [ ]:
# d) Gráfica accuracy vs número de vecinos
import matplotlib.pyplot as plt
acc_train = []
acc_test = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn.fit(X_train, y_train)
    acc_train.append(knn.score(X_train, y_train))
    acc_test.append(knn.score(X_test, y_test))
plt.plot(k_range, acc_train, label='Train')
plt.plot(k_range, acc_test, label='Test')
plt.xlabel('Número de vecinos (k)')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy vs Número de vecinos')
plt.show()

In [ ]:
# e) Experimentos con distancia de Minkowski para p=1,2,10
for p in [1, 2, 10]:
    knn = KNeighborsClassifier(n_neighbors=k_opt, metric='minkowski', p=p)
    knn.fit(X_train, y_train)
    acc = knn.score(X_test, y_test)
    print(f'Accuracy test con p={p}: {acc:.4f}')

### f) Pregunta teórica: efecto de p en Minkowski
**Respuesta:** p=1 corresponde a la distancia Manhattan, p=2 a Euclídea, p=10 se aproxima a la distancia Chebyshev. El valor óptimo depende de la estructura de los datos.

## 3. Selección de atributos
En esta sección se exploran métodos de reducción de dimensionalidad y su impacto en KNN.

In [ ]:
# a) VarianceThreshold
from sklearn.feature_selection import VarianceThreshold
u = 0.1
vt = VarianceThreshold(threshold=u)
X_train_vt = vt.fit_transform(X_train)
X_test_vt = vt.transform(X_test)
print(f'Número de atributos tras VarianceThreshold: {X_train_vt.shape[1]}')

In [ ]:
# b) Gráfica accuracy vs umbral u
us = np.linspace(0, 1, 20)
accs = []
for u in us:
    vt = VarianceThreshold(threshold=u)
    X_train_vt = vt.fit_transform(X_train)
    X_test_vt = vt.transform(X_test)
    knn = KNeighborsClassifier(n_neighbors=k_opt)
    knn.fit(X_train_vt, y_train)
    accs.append(knn.score(X_test_vt, y_test))
plt.plot(us, accs)
plt.xlabel('Umbral de varianza (u)')
plt.ylabel('Accuracy en test')
plt.title('Accuracy vs Umbral de varianza')
plt.show()

### c) Pregunta teórica: sentido de u=0 y u=1
**Respuesta:** u=0 no elimina ningún atributo (salvo los constantes). u=1 elimina todos los atributos salvo los que tengan varianza máxima, lo que normalmente deja muy pocos o ninguno.

In [ ]:
# d) SelectKBest
from sklearn.feature_selection import SelectKBest, f_classif
K = 10
selector = SelectKBest(score_func=f_classif, k=K)
X_train_kbest = selector.fit_transform(X_train, y_train)
X_test_kbest = selector.transform(X_test)
print(f'Número de atributos seleccionados: {X_train_kbest.shape[1]}')

In [ ]:
# e) Búsqueda del mejor K (número de atributos)
Ks = range(1, X_train.shape[1]+1)
accs_k = []
for K in Ks:
        selector = SelectKBest(score_func=f_classif, k=K)
        X_train_kbest = selector.fit_transform(X_train, y_train)
        X_test_kbest = selector.transform(X_test)
        knn = KNeighborsClassifier(n_neighbors=k_opt)
        knn.fit(X_train_kbest, y_train)
        accs_k.append(knn.score(X_test_kbest, y_test))
plt.plot(Ks, accs_k)
plt.xlabel('Número de atributos (K)')
plt.ylabel('Accuracy en test')
plt.title('Accuracy vs Número de atributos (SelectKBest)')
plt.show()
K_best = Ks[np.argmax(accs_k)]
        print(f'Mejor K: {K_best}')

In [ ]:
# f) mRMR (a completar en mRMR.py)
from mRMR import mRMR_selection
# Suponiendo que mRMR_selection(X, y, K) devuelve los índices de los K mejores atributos
K = 10
idx_mrmr = mRMR_selection(X_train, y_train, K)
X_train_mrmr = X_train[:, idx_mrmr]
X_test_mrmr = X_test[:, idx_mrmr]
knn = KNeighborsClassifier(n_neighbors=k_opt)
knn.fit(X_train_mrmr, y_train)
acc_mrmr = knn.score(X_test_mrmr, y_test)
print(f'Accuracy con mRMR y K={K}: {acc_mrmr:.4f}')

### g) Pregunta teórica: mejor valor de k
**Respuesta:** El mejor valor de k depende del conjunto de datos y debe determinarse empíricamente mediante validación cruzada.

### h) Pregunta teórica: papel de la información mutua en mRMR
**Respuesta:** La información mutua mide la dependencia entre variables. En mRMR se usa para seleccionar atributos relevantes y no redundantes. Se podría sustituir por otras métricas de dependencia, como la correlación.

### i) Pregunta teórica: comparación de métodos de selección
**Respuesta:** mRMR suele ser mejor que métodos univariados como SelectKBest porque considera la redundancia entre atributos, pero es más costoso computacionalmente.